In [1]:
import os
import tools
import numpy as np
import pandas as pd
import scanpy as sc

sc.logging.print_header()

scanpy==1.8.1 anndata==0.7.6 umap==0.5.1 numpy==1.22.2 scipy==1.6.2 pandas==1.2.4 scikit-learn==0.24.2 statsmodels==0.12.2 python-igraph==0.9.1 pynndescent==0.5.2


In [2]:
os.chdir('/lustre/scratch117/cellgen/team297/kt16/COVID_imperial_renal')

In [3]:
adata = sc.read_h5ad('h5ad/df.fil3_gex_bcells_no_CLL_vdj.h5ad')
adata

AnnData object with n_obs × n_vars = 70501 × 318
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'nCount_ADT_raw', 'nFeature_ADT_raw', 'nCount_RNA_raw', 'nFeature_RNA_raw', 'soupxOnAdt', 'soupxOnRna', 'percentMito', 'exclude', 'centre', 'pool', 'tcrId', 'bcrId', 'celltypist_broad_predLabel', 'celltypist_broad_maxPredProb', 'haniffa_broad_predLabel', 'haniffa_broad_maxPredProb', 'yoshida_broad_predLabel', 'yoshida_broad_maxPredProb', 'celltypist_detailed_predLabel', 'celltypist_detailed_maxPredProb', 'yoshida_detailed_predLabel', 'yoshida_detailed_maxPredProb', 'seurat_clusters', 'RNA_sequencing_saturation', 'pool_factor', 'RNA_readsConfidentlyOnTranscriptome', 'RNA_readsConfidentlyOnGenome', 'RNA_readsOnGenome', 'RNA_Q30readsBarcode', 'RNA_Q30readsRead', 'RNA_Q30readsUmi', 'rna_snn_res.4', 'rna_snn_res.0.5', 'rna_snn_res.0.1', 'harmony_snn_res.4', 'yoshida_broad_predLabel_majorVote', 'harmony_snn_res.0.5', 'harmony_snn_res.0.1', 'cell_type_compartment

In [4]:
# keep to good cells
adata = adata[(adata.obs['contig_QC_pass'] == 'True') & (adata.obs['merged_souporcell_status'] == 'singlet') & (adata.obs['celltype_B_corrected'] != 'doublet') & (adata.obs['sample_id'] != 'NA') & (~adata.obs['patient_id'].isin(['NULL', 'unresolved']))].copy()
adata

AnnData object with n_obs × n_vars = 44394 × 318
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'nCount_ADT_raw', 'nFeature_ADT_raw', 'nCount_RNA_raw', 'nFeature_RNA_raw', 'soupxOnAdt', 'soupxOnRna', 'percentMito', 'exclude', 'centre', 'pool', 'tcrId', 'bcrId', 'celltypist_broad_predLabel', 'celltypist_broad_maxPredProb', 'haniffa_broad_predLabel', 'haniffa_broad_maxPredProb', 'yoshida_broad_predLabel', 'yoshida_broad_maxPredProb', 'celltypist_detailed_predLabel', 'celltypist_detailed_maxPredProb', 'yoshida_detailed_predLabel', 'yoshida_detailed_maxPredProb', 'seurat_clusters', 'RNA_sequencing_saturation', 'pool_factor', 'RNA_readsConfidentlyOnTranscriptome', 'RNA_readsConfidentlyOnGenome', 'RNA_readsOnGenome', 'RNA_Q30readsBarcode', 'RNA_Q30readsRead', 'RNA_Q30readsUmi', 'rna_snn_res.4', 'rna_snn_res.0.5', 'rna_snn_res.0.1', 'harmony_snn_res.4', 'yoshida_broad_predLabel_majorVote', 'harmony_snn_res.0.5', 'harmony_snn_res.0.1', 'cell_type_compartment

In [8]:
# calculate days to sampling, days to admission, days to first symptoms, days to discharge from positive swab
tmp = adata.copy()
tmp.obs['sample_date'] = tmp.obs['sample_date'].astype('object')
tmp.obs['date_first_symptoms'] = tmp.obs['date_first_symptoms'].astype('object')
tmp.obs['date_positive_swab'] = tmp.obs['date_positive_swab'].astype('object')
tmp.obs['date_first_symptoms'].replace({'NA':np.nan}, inplace = True)
tmp.obs['date_positive_swab'].replace({'NA':np.nan}, inplace = True)
time_from_first_symptoms = pd.to_datetime(tmp.obs['sample_date'], format='%d/%m/%Y') - pd.to_datetime(tmp.obs['date_first_symptoms'], format='%d/%m/%Y')
time_from_positive_swab = pd.to_datetime(tmp.obs['sample_date'], format='%d/%m/%Y') - pd.to_datetime(tmp.obs['date_positive_swab'], format='%d/%m/%Y')
days_from_first_symptoms = time_from_first_symptoms.dt.days
days_from_positive_swab = time_from_positive_swab.dt.days
days_from_first_symptoms.fillna(0, inplace = True)
days_from_positive_swab.fillna(0, inplace = True)
days_from_infection = [max(a,b) for a,b in zip(days_from_first_symptoms, days_from_positive_swab)]
adata.obs['time_from_first_symptoms'] = days_from_first_symptoms
adata.obs['time_from_positive_swab'] = days_from_positive_swab
adata.obs['time_from_infection'] = days_from_infection

/nfs/users/nfs_k/kt16/miniconda2/envs/dandelion/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: modifications to a property of a datetimelike object are not supported and are discarded. Change values on the original.
  return super().fillna(


In [10]:
adata.obs['centre']

UK-CIC10689738+UK-CIC10690026_AACACGTGTTGGTGGA-1          NCL
UK-CIC10689738+UK-CIC10690026_AACCATGCAGTCACTA-1          NCL
UK-CIC10689738+UK-CIC10690026_AACCGCGAGTCTCCTC-1          NCL
UK-CIC10689738+UK-CIC10690026_AACCGCGGTCCAACTA-1          NCL
UK-CIC10689738+UK-CIC10690026_AACGTTGCATCCCATC-1          NCL
                                                      ...    
UK-CIC10690384+UK-CIC10690576_TTTACTGCAATCTACG-1    Cambridge
UK-CIC10690384+UK-CIC10690576_TTTATGCGTCATCCCT-1    Cambridge
UK-CIC10690384+UK-CIC10690576_TTTCCTCTCATGTCCC-1    Cambridge
UK-CIC10690384+UK-CIC10690576_TTTGGTTGTACGAAAT-1    Cambridge
UK-CIC10690384+UK-CIC10690576_TTTGGTTGTTGTCTTT-1    Cambridge
Name: centre, Length: 44394, dtype: category
Categories (2, object): ['Cambridge', 'NCL']

In [9]:
# save the obs
adata.obs.to_csv('h5ad/vdj_obs.csv')

In [5]:
import dandelion as ddl
ddl.logging.print_header()

dandelion==0.1.13.dev2 pandas==1.2.4 numpy==1.22.2 matplotlib==3.3.4 networkx==2.5.1 scipy==1.6.2 skbio==0.5.6


In [6]:
vdj = ddl.read_h5ddl('h5ddl/df.fil3_gex_bcells_no_CLL_vdj.h5ddl')
vdj

Dandelion class object with n_obs = 45992 and n_contigs = 91813
    data: 'sequence_id', 'sequence', 'rev_comp', 'productive', 'v_call', 'd_call', 'j_call', 'sequence_alignment', 'germline_alignment', 'junction', 'junction_aa', 'v_cigar', 'd_cigar', 'j_cigar', 'stop_codon', 'vj_in_frame', 'locus', 'junction_length', 'np1_length', 'np2_length', 'v_sequence_start', 'v_sequence_end', 'v_germline_start', 'v_germline_end', 'd_sequence_start', 'd_sequence_end', 'd_germline_start', 'd_germline_end', 'j_sequence_start', 'j_sequence_end', 'j_germline_start', 'j_germline_end', 'v_score', 'v_identity', 'v_support', 'd_score', 'd_identity', 'd_support', 'j_score', 'j_identity', 'j_support', 'fwr1', 'fwr2', 'fwr3', 'fwr4', 'cdr1', 'cdr2', 'cdr3', 'cell_id', 'c_call', 'consensus_count', 'umi_count', 'v_call_10x', 'd_call_10x', 'j_call_10x', 'junction_10x', 'junction_10x_aa', 'v_call_genotyped', 'germline_alignment_d_mask', 'sample_id', 'c_sequence_alignment', 'c_germline_alignment', 'c_sequence_star

In [7]:
# keep only cells in the bdata object
bcr = vdj.data[vdj.data.cell_id.isin(adata.obs_names)]
bcr.shape

(88702, 84)

In [8]:
# transfer sample_id
sample = dict(zip(adata.obs_names, adata.obs['sample_id']))
bcr['sample_id'] = [sample[x] for x in bcr['cell_id']]

<ipython-input-8-e9f51e139801>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
vdj = ddl.Dandelion(bcr)
vdj

Dandelion class object with n_obs = 44394 and n_contigs = 88702
    data: 'sequence_id', 'sequence', 'rev_comp', 'productive', 'v_call', 'd_call', 'j_call', 'sequence_alignment', 'germline_alignment', 'junction', 'junction_aa', 'v_cigar', 'd_cigar', 'j_cigar', 'stop_codon', 'vj_in_frame', 'locus', 'junction_length', 'np1_length', 'np2_length', 'v_sequence_start', 'v_sequence_end', 'v_germline_start', 'v_germline_end', 'd_sequence_start', 'd_sequence_end', 'd_germline_start', 'd_germline_end', 'j_sequence_start', 'j_sequence_end', 'j_germline_start', 'j_germline_end', 'v_score', 'v_identity', 'v_support', 'd_score', 'd_identity', 'd_support', 'j_score', 'j_identity', 'j_support', 'fwr1', 'fwr2', 'fwr3', 'fwr4', 'cdr1', 'cdr2', 'cdr3', 'cell_id', 'c_call', 'consensus_count', 'umi_count', 'v_call_10x', 'd_call_10x', 'j_call_10x', 'junction_10x', 'junction_10x_aa', 'v_call_genotyped', 'germline_alignment_d_mask', 'sample_id', 'c_sequence_alignment', 'c_germline_alignment', 'c_sequence_star

In [11]:
# ddl.tl.find_clones(vdj)

In [12]:
from collections import defaultdict

In [14]:
vdjs = defaultdict(dict)
for x in list(set(bcr.sample_id)):
    vdjs[x] = ddl.Dandelion(bcr[bcr['sample_id'] == x])

In [15]:
from tqdm import tqdm

In [16]:
result_df = {}
for x in tqdm(vdjs):
    ddl.tl.find_clones(vdjs[x])
    result_df[x] = ddl.tl.clone_diversity(vdjs[x], groupby = 'sample_id', update_obs_meta = False, use_contracted = True, expanded_only = True)

Finding clones based on VDJ chains : 100%|██████████| 114/114 [00:00<00:00, 5417.40it/s]

Refining clone assignment based on VJ chain pairing : 100%|██████████| 317/317 [00:00<00:00, 474177.73it/s]


Computing Gini indices for cluster and vertex size using network.


Finding clones based on VDJ chains : 100%|██████████| 31/31 [00:00<00:00, 3271.44it/s]

Refining clone assignment based on VJ chain pairing : 100%|██████████| 63/63 [00:00<00:00, 293275.42it/s]
/nfs/users/nfs_k/kt16/miniconda2/envs/dandelion/lib/python3.8/site-packages/dandelion/tools/_diversity.py:464: UserWarning: The minimum cell numbers when grouped by sample_id is 63 (group C212_2705). Exercise caution when interpreting diversity measures.


Computing Gini indices for cluster and vertex size using network.


Finding clones based on VDJ chains : 100%|██████████| 59/59 [00:00<00:00, 3319.52it/s]

Refining clone assignment based on VJ chain pairing : 100%|██████████| 146/146 [00:00<00:00, 286287.23it/s]


...

Finding clones based on VDJ chains : 100%|██████████| 94/94 [00:00<00:00, 3776.30it/s]

Refining clone assignment based on VJ chain pairing : 100%|██████████| 235/235 [00:00<00:00, 363981.33it/s]


Computing Gini indices for cluster and vertex size using network.


Finding clones based on VDJ chains : 100%|██████████| 119/119 [00:00<00:00, 3753.02it/s]

Refining clone assignment based on VJ chain pairing : 100%|██████████| 350/350 [00:00<00:00, 336467.20it/s]


Computing Gini indices for cluster and vertex size using network.


100%|██████████| 187/187 [27:10<00:00,  8.72s/it]


In [17]:
result_df2 = {}
for x in tqdm(vdjs):
    result_df2[x] = ddl.tl.clone_diversity(vdjs[x], groupby = 'sample_id', update_obs_meta = False, expanded_only = True)

  0%|          | 0/187 [00:00<?, ?it/s]

Computing Gini indices for cluster and vertex size using network.


  1%|          | 1/187 [00:07<24:19,  7.85s/it]/nfs/users/nfs_k/kt16/miniconda2/envs/dandelion/lib/python3.8/site-packages/dandelion/tools/_diversity.py:464: UserWarning: The minimum cell numbers when grouped by sample_id is 63 (group C212_2705). Exercise caution when interpreting diversity measures.


Computing Gini indices for cluster and vertex size using network.


  1%|          | 2/187 [00:12<17:59,  5.83s/it]

...

 99%|█████████▉| 185/187 [24:13<00:52, 26.23s/it]

Computing Gini indices for cluster and vertex size using network.


 99%|█████████▉| 186/187 [24:19<00:20, 20.25s/it]

Computing Gini indices for cluster and vertex size using network.


100%|██████████| 187/187 [24:27<00:00,  7.85s/it]


In [21]:
result_df3 = {}
for x in tqdm(vdjs):
    ddl.tl.generate_network(vdjs[x], verbose = False)
    ddl.tl.clone_centrality(vdjs[x], verbose = False)
    result_df3[x] = ddl.tl.clone_diversity(vdjs[x], groupby = 'sample_id', update_obs_meta = False, expanded_only = True, metric = 'clone_centrality', reconstruct_network = False)

  0%|          | 0/187 [00:00<?, ?it/s]

Computing gini indices for clone size using metadata and node closeness centrality using network.


  1%|          | 1/187 [00:07<22:48,  7.36s/it]/nfs/users/nfs_k/kt16/miniconda2/envs/dandelion/lib/python3.8/site-packages/dandelion/tools/_diversity.py:464: UserWarning: The minimum cell numbers when grouped by sample_id is 63 (group C212_2705). Exercise caution when interpreting diversity measures.


Computing gini indices for clone size using metadata and node closeness centrality using network.


  1%|          | 2/187 [00:11<17:22,  5.64s/it]

...

 99%|█████████▉| 185/187 [23:27<00:23, 11.56s/it]

Computing gini indices for clone size using metadata and node closeness centrality using network.


 99%|█████████▉| 186/187 [23:32<00:09,  9.73s/it]

Computing gini indices for clone size using metadata and node closeness centrality using network.


100%|██████████| 187/187 [23:39<00:00,  7.59s/it]


In [22]:
results1 = pd.concat(result_df)
results1.index = [x[0] for x in results1.index]
results1

,clone_network_cluster_size_gini,clone_network_vertex_size_gini
C101_200121,0.701514,0.108485
C212_2705,0.250000,0.000000
C193_2605,0.606061,0.064935
C40_0105,0.000000,0.250000
C140_300420,0.177833,0.053472
...,...,...
C137_190121,0.206306,0.069444
C140_300121,0.152865,0.041828
C106_2204,0.083333,0.000000
C128_1105,0.232143,0.122312


In [23]:
results2 = pd.concat(result_df2)
results2.index = [x[0] for x in results2.index]
results2

,clone_network_cluster_size_gini,clone_network_vertex_size_gini
C101_200121,0.291235,0.108485
C212_2705,0.250000,0.000000
C193_2605,0.476190,0.064935
C40_0105,0.000000,0.250000
C140_300420,0.061207,0.053472
...,...,...
C137_190121,0.063333,0.069444
C140_300121,0.046394,0.041828
C106_2204,0.083333,0.000000
C128_1105,0.144009,0.122312


In [24]:
results3 = pd.concat(result_df3)
results3.index = [x[0] for x in results3.index]
results3

,clone_size_gini,clone_centrality_gini
C101_200121,0.126297,0.611111
C212_2705,0.060890,0.000000
C193_2605,0.040710,0.214286
C40_0105,0.010949,0.000000
C140_300420,0.061128,0.350000
...,...,...
C137_190121,0.045464,0.400000
C140_300121,0.092574,0.372587
C106_2204,0.044440,0.000000
C128_1105,0.137144,0.408602


In [25]:
cell_counts = {}
for x in result_df:
    cell_counts[x] = vdjs[x].metadata.shape[0]
cell_counts = pd.DataFrame(cell_counts, index = ['cell_counts']).T
cell_counts

,cell_counts
C101_200121,317
C212_2705,63
C193_2605,146
C40_0105,273
C140_300420,989
...,...
C137_190121,675
C140_300121,1329
C106_2204,259
C128_1105,235


In [33]:
cell_counts2 = {}
for x in result_df:
    cell_counts2[x] = len(vdjs[x].graph[1])
cell_counts2 = pd.DataFrame(cell_counts2, index = ['cell_counts']).T
cell_counts2

,cell_counts
C101_200121,63
C212_2705,6
C193_2605,7
C40_0105,3
C140_300420,120
...,...
C137_190121,60
C140_300121,259
C106_2204,22
C128_1105,62


In [34]:
results1['cell_counts'] = cell_counts
results1

,clone_network_cluster_size_gini,clone_network_vertex_size_gini,cell_counts
C101_200121,0.701514,0.108485,317
C212_2705,0.250000,0.000000,63
C193_2605,0.606061,0.064935,146
C40_0105,0.000000,0.250000,273
C140_300420,0.177833,0.053472,989
...,...,...,...
C137_190121,0.206306,0.069444,675
C140_300121,0.152865,0.041828,1329
C106_2204,0.083333,0.000000,259
C128_1105,0.232143,0.122312,235


In [35]:
results2['cell_counts'] = cell_counts
results2

,clone_network_cluster_size_gini,clone_network_vertex_size_gini,cell_counts
C101_200121,0.291235,0.108485,317
C212_2705,0.250000,0.000000,63
C193_2605,0.476190,0.064935,146
C40_0105,0.000000,0.250000,273
C140_300420,0.061207,0.053472,989
...,...,...,...
C137_190121,0.063333,0.069444,675
C140_300121,0.046394,0.041828,1329
C106_2204,0.083333,0.000000,259
C128_1105,0.144009,0.122312,235


In [36]:
results3['cell_counts'] = cell_counts
results3

,clone_size_gini,clone_centrality_gini,cell_counts
C101_200121,0.126297,0.611111,317
C212_2705,0.060890,0.000000,63
C193_2605,0.040710,0.214286,146
C40_0105,0.010949,0.000000,273
C140_300420,0.061128,0.350000,989
...,...,...,...
C137_190121,0.045464,0.400000,675
C140_300121,0.092574,0.372587,1329
C106_2204,0.044440,0.000000,259
C128_1105,0.137144,0.408602,235


In [37]:
results1['cell_counts_expanded'] = cell_counts2
results1

,clone_network_cluster_size_gini,clone_network_vertex_size_gini,cell_counts,cell_counts_expanded
C101_200121,0.701514,0.108485,317,63
C212_2705,0.250000,0.000000,63,6
C193_2605,0.606061,0.064935,146,7
C40_0105,0.000000,0.250000,273,3
C140_300420,0.177833,0.053472,989,120
...,...,...,...,...
C137_190121,0.206306,0.069444,675,60
C140_300121,0.152865,0.041828,1329,259
C106_2204,0.083333,0.000000,259,22
C128_1105,0.232143,0.122312,235,62


In [38]:
results2['cell_counts_expanded'] = cell_counts2
results2

,clone_network_cluster_size_gini,clone_network_vertex_size_gini,cell_counts,cell_counts_expanded
C101_200121,0.291235,0.108485,317,63
C212_2705,0.250000,0.000000,63,6
C193_2605,0.476190,0.064935,146,7
C40_0105,0.000000,0.250000,273,3
C140_300420,0.061207,0.053472,989,120
...,...,...,...,...
C137_190121,0.063333,0.069444,675,60
C140_300121,0.046394,0.041828,1329,259
C106_2204,0.083333,0.000000,259,22
C128_1105,0.144009,0.122312,235,62


In [39]:
results3['cell_counts_expanded'] = cell_counts2
results3

,clone_size_gini,clone_centrality_gini,cell_counts,cell_counts_expanded
C101_200121,0.126297,0.611111,317,63
C212_2705,0.060890,0.000000,63,6
C193_2605,0.040710,0.214286,146,7
C40_0105,0.010949,0.000000,273,3
C140_300420,0.061128,0.350000,989,120
...,...,...,...,...
C137_190121,0.045464,0.400000,675,60
C140_300121,0.092574,0.372587,1329,259
C106_2204,0.044440,0.000000,259,22
C128_1105,0.137144,0.408602,235,62


In [40]:
results1.to_csv('h5ddl/network_gini.csv')

In [41]:
results2.to_csv('h5ddl/network_gini_not_contracted.csv')

In [42]:
results3.to_csv('h5ddl/network_gini_centrality.csv')